In [4]:
from torch_geometric.data import Data
from DataBuilder import DataBuilder
from constants import TRIPLES_PATH, ENTITIES_PATH, RELATIONS_PATH
from LLMEncoder import LLMEncoder

In [5]:
from transformers import RobertaTokenizer, RobertaModel
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaModel.from_pretrained("roberta-base")
llm_encoder = LLMEncoder(tokenizer, model)

data_builder = DataBuilder(TRIPLES_PATH, ENTITIES_PATH, RELATIONS_PATH, llm_encoder=llm_encoder)
data_builder.get_edge_index()

tensor([[ 219,  212,  223,  ..., 4345, 2269, 3217],
        [1950, 3510, 2846,  ..., 5495, 5495, 5495]])

In [ ]:
data = Data(x=data_builder.x, edge_index=data_builder.edge_index, ...)